# GMRES

In [1]:
import numpy as np
from numpy.linalg import qr

from functions import arnoldi, back_substitution

from scipy.sparse.linalg import lsqr
from scipy.linalg import solve_triangular

For some reason I cannot understand, the following code did not work with 

y = solve_triangular(R[:-1, :] , beta*Q[0][:-1], lower = True).

Tried other solvers like scipy.sparse.linalg.lsqr and it worked. So I decided to implement back substitution manually.

In [2]:
def GMRES(A, b, x0, k_max = None , epsilon = 1e-8):
    """
    Generalized Minimal RESidual method for solving linear systems.
    
    Parameters:
    -----------
    A : numpy.ndarray
        Coefficient matrix of the linear system.
        
    b : numpy.ndarray
        Right-hand side vector of the linear system.
        
    x0 : numpy.ndarray
        Initial guess for the solution.
        
    k_max : int
        Maximum number of iterations.
        
    epsilon : float, optional
        Tolerance for convergence.
    
    Returns:
    --------
    numpy.ndarray
        Approximate solution to the linear system.
    """
    
    n = A.shape[0]
    
    if (k_max is None):
        k_max = n
        
    elif k_max > n:
        k_max = n
        
    r0 = b - A @ x0
    p0 = np.linalg.norm(r0)
    beta = p0
    pk = p0
    k = 0
    
    while pk > epsilon*p0 and k < k_max:
        
        k += 1
        
        V, H = arnoldi(A, b, x0, k) # Arnoldi algorithm to generate V_{k+1} and H_{K+1, K}
        
        Q, R = qr(H, mode = 'complete')
        
        pk = abs(beta*Q[0, k]) # Compute norm of residual vector
        
        yk = back_substitution(R[:-1, :] , beta*Q[0][:-1])
    
        xk = x0 + V[:, :-1]@yk # Compute the new approximation x0 + V_{k}y
        
    return xk, pk

# Unit tests

#### Simple systems. The results are the correct ones.

In [3]:
A = np.array([[1, 1], [3,4]])
b = np.array([3, 2])
x0 = np.array([1, 2])

max_iter = 5

x = GMRES(A, b, x0) # Not specifying max_iter it takes n = 2 iterations

x

(array([10., -7.]), 0.0)

In [4]:
A = np.array([[1,0,0],[0,2,0],[0,0,3]])
# A = lambda x: a.dot(x)
b = np.array([1, 4, 6])
x0 = np.zeros(b.size)
GMRES(A, b, x0, 5)

(array([1., 2., 2.]), 1.098089075328358e-16)

#### Comparison with Scipy GMRES implementation

In [5]:
import scipy.sparse.linalg as spla

A = np.array([[1, 1, 4, 9], [3, 4, 6, 9], [4, 1, 1, 3], [3, 2, 1, 1]])
b = np.array([3, 2, 2, -3])
x0 = np.array([0, 0, 1, 0])
spla.gmres(A,b,x0, restart = None)[0], GMRES(A, b, x0, 4) # Converges in n = 4 iterations! God

(array([ 2.96296296, -9.44444444, 10.7037037 , -3.7037037 ]),
 (array([ 2.96296296, -9.44444444, 10.7037037 , -3.7037037 ]),
  1.0997010364081157e-14))

In [7]:
import scipy.io
import scipy.sparse

# Load the .mtx file
A = scipy.io.mmread("data/bcsstk18.mtx")

# Convert the matrix to CRS format
A = scipy.sparse.csr_matrix(A)

# https://sparse.tamu.edu/HB/bcsstk18

# Problem type: structural problem - S STIFFNESS MATRIX arised from R.E.GINNA NUCLEAR POWER STATION
# Dimension: 11,948 x 11,948
# 149,090 nonzero elements
# Condition number: 3.459995e+11

b = np.ones(A.shape[0])
x0 = np.zeros(b.size)

x = spla.gmres(A, b, x0, restart = 100, maxiter = 1000)[0]
residual_calculated1 = np.linalg.norm(A@x - b)

print("Calculated Scipy residual with Ax-b:", residual_calculated1)

x = GMRES(A, b, x0, restart = 100, maxiter = 1000)[0]
residual_calculated2 = np.linalg.norm(A@x - b)

print("\nOur implementation residual with Ax-b:", residual_calculated2)

KeyboardInterrupt: 

Tried different restart values and more iterations and the residual consistently decreases, but it takes too long.